In [17]:
import random
import numpy as np
import json
import os

from problem import TSPProblem, KnapsackProblem
from GA.crossover_operators import order_crossover, pmx_crossover, cycle_crossover, arithmetic_crossover, blend_crossover
from GA.mutation_operators import swap_mutation, gaussian_mutation, uniform_mutation
from GA.selection_operators import tournament_selection
from experiment import Experiment, MFEAExperiment, NumpyEncoder

In [18]:
random.seed(42)
np.random.seed(42)
results = {}

In [19]:
tsp = TSPProblem(file_path="dataset/tsp/eil51.tsp", encoding='permutation')

crossover_ops = {
    "order": order_crossover,
    "pmx": pmx_crossover,
    "cycle": cycle_crossover
}

task1 = {}
for name, op in crossover_ops.items():
    exp = Experiment(
        problem=tsp,
        seeds=range(5),
        pop_size=100,
        generations=500,
        crossover_prob=0.9,
        mutation_prob=0.1,
        crossover_op=op,
        mutation_op=swap_mutation,
        selection_op=tournament_selection,
        elitism=2
    )
    exp.run(verbose=False)
    task1[name] = -exp.stats["max"]
results["task1"] = task1

Loaded TSP problem with 51 cities from dataset/tsp/eil51.tsp


Running tsp experiment: 100%|██████████| 5/5 [00:11<00:00,  2.26s/it]


✓ Results saved to results/ga/order/swap/tournament/tsp


Running tsp experiment: 100%|██████████| 5/5 [00:08<00:00,  1.72s/it]


✓ Results saved to results/ga/pmx/swap/tournament/tsp


Running tsp experiment: 100%|██████████| 5/5 [00:09<00:00,  1.93s/it]

✓ Results saved to results/ga/cycle/swap/tournament/tsp


In [20]:
tsp_real = TSPProblem(file_path="dataset/tsp/eil51.tsp", encoding='real')

real_ops = {
    "arithmetic": (arithmetic_crossover, gaussian_mutation),
    "blx_alpha": (blend_crossover, uniform_mutation)
}

task2 = {}
for name, (c_op, m_op) in real_ops.items():
    exp = Experiment(
        problem=tsp_real,
        seeds=range(5),
        pop_size=100,
        generations=500,
        crossover_prob=0.9,
        mutation_prob=0.1,
        crossover_op=c_op,
        mutation_op=m_op,
        selection_op=tournament_selection,
        elitism=2
    )
    exp.run(verbose=False)
    task2[name] = -exp.stats["max"]
results["task2"] = task2


Loaded TSP problem with 51 cities from dataset/tsp/eil51.tsp


Running tsp experiment: 100%|██████████| 5/5 [00:08<00:00,  1.75s/it]


✓ Results saved to results/ga/arithmetic/gaussian/tournament/tsp


Running tsp experiment: 100%|██████████| 5/5 [00:09<00:00,  1.91s/it]

✓ Results saved to results/ga/blend/uniform/tournament/tsp


In [21]:
tsp51 = TSPProblem(file_path="dataset/tsp/eil51.tsp", encoding='permutation')
tsp101 = TSPProblem(file_path="dataset/tsp/eil101.tsp", encoding='permutation')

mfea_exp = MFEAExperiment(
    problems=[tsp51, tsp101],
    seeds=range(5),
    pop_size=100,
    generations=500,
    rmp=0.3,
    cross_crossover_op=order_crossover
)
mfea_exp.run(verbose=False)

results["task3"] = {
    "tsp51": mfea_exp.stats["task_0"]["best_tour_length"],
    "tsp101": mfea_exp.stats["task_1"]["best_tour_length"]
}


Loaded TSP problem with 51 cities from dataset/tsp/eil51.tsp
Loaded TSP problem with 101 cities from dataset/tsp/eil101.tsp


Running MFEA on order_tournament_tsp_tsp: 100%|██████████| 5/5 [00:42<00:00,  8.46s/it]


In [22]:
knapsack = KnapsackProblem(file_prefix="dataset/knapsack/p07", encoding='real')

mfea_exp2 = MFEAExperiment(
    problems=[tsp51, knapsack],
    seeds=range(5),
    pop_size=100,
    generations=500,
    rmp=0.3,
    cross_crossover_op=arithmetic_crossover
)
mfea_exp2.run(verbose=False)

results["task4"] = {
    "tsp51": mfea_exp2.stats["task_0"]["best_tour_length"],
    "knapsack": mfea_exp2.stats["task_1"]["max"]
}


Loaded Knapsack: 15 items, capacity=750


Running MFEA on arithmetic_tournament_tsp_knapsack: 100%|██████████| 5/5 [00:18<00:00,  3.70s/it]


In [23]:
os.makedirs("results/simple", exist_ok=True)
with open("results/simple/results.json", "w") as f:
    json.dump(results, f, indent=4, cls=NumpyEncoder)
    
print("Summary results:", results)


Summary results: {'task1': {'order': np.float64(473.42066186107525), 'pmx': np.float64(615.1183795018541), 'cycle': np.float64(571.3011433873755)}, 'task2': {'arithmetic': np.float64(557.1060738340791), 'blx_alpha': np.float64(971.5536188311206)}, 'task3': {'tsp51': np.float64(511.78957449484307), 'tsp101': np.float64(973.1986273165792)}, 'task4': {'tsp51': np.float64(471.3120525815979), 'knapsack': np.int64(1458)}}
